In [1]:
import pandas as pd

In [78]:
df = pd.read_csv("train.csv")
# print(df.info())
df.isnull().sum()
# df.describe()
# df.shape
# df.duplicated().sum()
# df.unique()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
# Test File

In [171]:
df_test = pd.read_csv("test.csv")
# print(df_test.info())
# df_test.isnull().sum()
# df_test.describe()
# df_test.shape
# df_test.duplicated().sum()
# df_test.unique()

In [130]:
import numpy as np
numeric_cols = df.select_dtypes(include=[np.number])
skew_values = numeric_cols.skew()
# print(skew_values)

In [172]:

numeric_cols = df_test.select_dtypes(include=[np.number])
skew_values = numeric_cols.skew()
# print(skew_values)
# df_test.isnull().sum()
# print(df_test.info())


In [80]:
df["Age"] = df["Age"].fillna(df["Age"].mean())
# df["Embarked"].unique()
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])
df["Embarked"].isnull().sum()
# print(df.info())
df["Cabin"] = df["Cabin"].fillna("Unknown")
df["CabinDeck"] = df["Cabin"].apply(lambda x: x[0] if x != "Unknown" else "Unknown")
# df["CabinDeck"].unique()
# df["Cabin"] != "Unknown" # agr unknonw k braber nh hy tu wo true hoga
df["HasCabin"] = (df["Cabin"] != "Unknown").astype(int)

# Handled missing value age, Embarked and cabins
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
df["IsAlone"] = 0
df.loc[df["FamilySize"] == 1, "IsAlone"] = 1


# 1) extract Title (text before the dot, after a space)
df['Title'] = df['Name'].str.extract(r',\s*([^\.]+)\.', expand=False).str.strip()
# 2) standardize some titles
df['Title'] = df['Title'].replace({
    'Mlle': 'Miss',
    'Ms': 'Miss',
    'Mme': 'Mrs'
})
# 3) group rare titles into 'Rare'
rare_titles = ['Dr','Rev','Col','Major','Capt','Lady','Sir','Don','Dona','Countess','Jonkheer','the Countess']
df['Title'] = df['Title'].apply(lambda x: 'Rare' if x in rare_titles else x)


# now you can inspect unique titles and survival rates
print(df['Title'].unique())
print(df.groupby('Title')['Survived'].mean().sort_values(ascending=False))

df = pd.get_dummies(df, columns=['Title'], drop_first=True)


['Mr' 'Mrs' 'Miss' 'Master' 'Rare']
Title
Mrs       0.793651
Miss      0.702703
Master    0.575000
Rare      0.347826
Mr        0.156673
Name: Survived, dtype: float64


In [173]:
df_test["Age"] = df_test["Age"].fillna(df_test["Age"].mean())
# Fill missing value with median
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)
# df["Embarked"].unique()
df_test["Embarked"] = df_test["Embarked"].fillna(df_test["Embarked"].mode()[0])
df_test["Embarked"].isnull().sum()
# print(df.info())
df_test["Cabin"] = df_test["Cabin"].fillna("Unknown")
df_test["CabinDeck"] = df_test["Cabin"].apply(lambda x: x[0] if x != "Unknown" else "Unknown")
# df["CabinDeck"].unique()
# df["Cabin"] != "Unknown" # agr unknonw k braber nh hy tu wo true hoga
df_test["HasCabin"] = (df_test["Cabin"] != "Unknown").astype(int)

# Features engineering
df_test["Sex"] = df_test["Sex"].map({"male": 0, "female": 1})
df_test["FamilySize"] = df_test["SibSp"] + df_test["Parch"] + 1
df_test["IsAlone"] = 0
df_test.loc[df_test["FamilySize"] == 1, "IsAlone"] = 1


# 1) extract Title (text before the dot, after a space)
df_test['Title'] = df_test['Name'].str.extract(r',\s*([^\.]+)\.', expand=False).str.strip()
# 2) standardize some titles
df_test['Title'] = df_test['Title'].replace({
    'Mlle': 'Miss',
    'Ms': 'Miss',
    'Mme': 'Mrs'
})
# 3) group rare titles into 'Rare'
rare_titles = ['Dr','Rev','Col','Major','Capt','Lady','Sir','Don','Dona','Countess','Jonkheer','the Countess']
df_test['Title'] = df_test['Title'].apply(lambda x: 'Rare' if x in rare_titles else x)


# now you can inspect unique titles and survival rates
print(df_test['Title'].unique())
# print(df_test.groupby('Title')['Survived'].mean().sort_values(ascending=False))

df_test = pd.get_dummies(df_test, columns=['Title'], drop_first=True)
# print(df_test.info())


['Mr' 'Mrs' 'Miss' 'Master' 'Rare']


C:\Users\Faisal Zamir\AppData\Local\Temp\ipykernel_17952\226010754.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)


In [91]:
# df = df.drop(columns=['Name'])
# df = df.drop(columns=['PassengerId'])
# df = df.drop(columns=['Cabin'])
# df = df.drop(columns=['Ticket'])
# print(df.columns.value_counts())


In [184]:
df_test = df_test.drop(columns=['Name'])
df_test = df_test.drop(columns=['Cabin'])
df_test = df_test.drop(columns=['Ticket'])

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   PassengerId        418 non-null    int64   
 1   Pclass             418 non-null    category
 2   Sex                418 non-null    int64   
 3   Age                418 non-null    float64 
 4   SibSp              418 non-null    int64   
 5   Parch              418 non-null    int64   
 6   Fare               418 non-null    float64 
 7   Embarked           418 non-null    object  
 8   CabinDeck          418 non-null    object  
 9   HasCabin           418 non-null    int64   
 10  FamilySize         418 non-null    int64   
 11  IsAlone            418 non-null    int64   
 12  Title_Miss         418 non-null    int64   
 13  Title_Mr           418 non-null    int64   
 14  Title_Mrs          418 non-null    int64   
 15  Title_Rare         418 non-null    int64   
 16  Embarked

In [111]:
# 1. Pclass → convert to category
df["Pclass"] = df["Pclass"].astype("category")

# 4. HasCabin → ensure int64 (for consistency)
df["HasCabin"] = df["HasCabin"].astype("int64")

# 5. Title dummies → convert from bool → int64
df[["Title_Miss", "Title_Mr", "Title_Mrs", "Title_Rare"]] = \
    df[["Title_Miss", "Title_Mr", "Title_Mrs", "Title_Rare"]].astype("int64")

# Convert Embarked dummy columns from bool → int64
df[["Embarked_C", "Embarked_Q", "Embarked_S"]] = \
    df[["Embarked_C", "Embarked_Q", "Embarked_S"]].astype("int64")

# Convert CabinDeck dummy columns from bool → int64
df[["CabinDeck_A", "CabinDeck_B", "CabinDeck_C"]] = \
    df[["CabinDeck_A", "CabinDeck_B", "CabinDeck_C"]].astype("int64")

# Find all boolean columns
bool_cols = df.select_dtypes(include=["bool"]).columns

# Convert them to int64
df[bool_cols] = df[bool_cols].astype("int64")




In [185]:
# For Embarked
embarked_dummies = pd.get_dummies(df_test['Embarked'], prefix='Embarked')

# Reindex to match train columns
embarked_dummies = embarked_dummies.reindex(columns=['Embarked_C','Embarked_Q','Embarked_S'], fill_value=0)

# Add to df_test
df_test = pd.concat([df_test, embarked_dummies], axis=1)


In [186]:
# 1. Pclass → convert to category
df_test["Pclass"] = df_test["Pclass"].astype("category")

# 4. HasCabin → ensure int64 (for consistency)
df_test["HasCabin"] = df_test["HasCabin"].astype("int64")

# 5. Title dummies → convert from bool → int64
df_test[["Title_Miss", "Title_Mr", "Title_Mrs", "Title_Rare"]] = \
    df_test[["Title_Miss", "Title_Mr", "Title_Mrs", "Title_Rare"]].astype("int64")

cabin_dummies = pd.get_dummies(df_test['CabinDeck'], prefix='CabinDeck')
cabin_dummies = cabin_dummies.reindex(columns=['CabinDeck_A','CabinDeck_B','CabinDeck_C','CabinDeck_D',
                                               'CabinDeck_E','CabinDeck_F','CabinDeck_G','CabinDeck_T','CabinDeck_Unknown'],
                                      fill_value=0)
df_test = pd.concat([df_test, cabin_dummies], axis=1)


# Find all boolean columns
bool_cols = df_test.select_dtypes(include=["bool"]).columns

# Convert them to int64
df_test[bool_cols] = df_test[bool_cols].astype("int64")




In [117]:
# print(df.info())
# df.isnull().sum()
# df.describe()
# df.shape
# df.duplicated().sum()
# df[df.duplicated()]
dupes = df[df.duplicated(keep=False)]  # show all duplicates, not just first
# print(dupes.head(20))



In [187]:
# print(df_test.info())
# df_test.isnull().sum()
# df_test.describe()
# df_test.shape
# df_test.duplicated().sum()
# df_test[df_test.duplicated()]
# dupes = df_test[df_test.duplicated(keep=False)]  # show all duplicates, not just first
# print(dupes.head(20))

In [121]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[["Age", "Fare"]] = scaler.fit_transform(df[["Age", "Fare"]])
# df["Fare"]

In [197]:
df_test[["Age", "Fare"]] = scaler.fit_transform(df_test[["Age", "Fare"]])
# df_test["Fare"]

In [122]:
from sklearn.model_selection import train_test_split

X = df.drop("Survived", axis=1)
y = df["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [209]:
df_test_model = df_test.drop(columns=['PassengerId', 'CabinDeck', 'Embarked'], errors='ignore')
# Remove duplicate columns in test data
df_test_model = df_test_model.loc[:,~df_test_model.columns.duplicated()]

# Columns of training data used in model
print("Train columns:")
print(X_train.columns.tolist())

# Columns of test data after preprocessing
print("Test columns:")
print(df_test_model.columns.tolist())
# Check if column names are exactly same
print("Columns equal?", X_train.columns.equals(df_test_model.columns))

# Check which columns are missing in test
missing_in_test = set(X_train.columns) - set(df_test_model.columns)
print("Missing in test:", missing_in_test)

# Extra columns in test
extra_in_test = set(df_test_model.columns) - set(X_train.columns)
print("Extra in test:", extra_in_test)



Train columns:
['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'HasCabin', 'FamilySize', 'IsAlone', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'CabinDeck_A', 'CabinDeck_B', 'CabinDeck_C', 'CabinDeck_D', 'CabinDeck_E', 'CabinDeck_F', 'CabinDeck_G', 'CabinDeck_T', 'CabinDeck_Unknown']
Test columns:
['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'HasCabin', 'FamilySize', 'IsAlone', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'CabinDeck_A', 'CabinDeck_B', 'CabinDeck_C', 'CabinDeck_D', 'CabinDeck_E', 'CabinDeck_F', 'CabinDeck_G', 'CabinDeck_T', 'CabinDeck_Unknown']
Columns equal? True
Missing in test: set()
Extra in test: set()


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [124]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[98 12]
 [19 50]]
              precision    recall  f1-score   support

           0       0.84      0.89      0.86       110
           1       0.81      0.72      0.76        69

    accuracy                           0.83       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.83      0.83      0.82       179



In [211]:
# df_test
X_test_final = df_test_model  # after preprocessing, scaled and encoded

y_pred_test = model.predict(X_test_final)
submission = pd.DataFrame({
    "PassengerId": df_test["PassengerId"],  # original IDs from test file
    "Survived": y_pred_test
})

submission.to_csv("submission.csv", index=False)
